**Homework 3- Question 3 Police Problem Solution**

The given formulation for police problem in the question is as follows

\begin{equation*}
\begin{aligned}
\max_{x} \quad \{\min_{u} & \{u^TAx\} \}\\
\textrm{s.t.} \quad & u \geq 0 \\
  & x \geq 0    \\
  & e^Tx = e^Tu = 1 \\
\end{aligned}
\end{equation*}

We can rewrite the above formulation in a way that is simple to code in Pyomo (by introducing a new variable). Here is an equivalent formulation to the above problem.

\begin{equation*}
\begin{aligned}
\max_{x,z} \quad  z\\
\textrm{s.t.} \quad & ze -Ax \leq 0 \\
  & e^Tx = 1 \\
  & x \geq 0    \\
\end{aligned}
\end{equation*}

Now, let us code this in Pyomo


Importing Libraries

In [1]:
from pyomo.environ import *
import numpy as np

Defining the model, sets and varibles

In [3]:
# Defining the Model
model = ConcreteModel()

# Sets
model.I = RangeSet(1, 3)

# Define variables
model.x = Var(model.I, domain=NonNegativeReals)
model.z = Var(domain = Reals)

Defining objective and constraints

In [ ]:
# Objective
model.obj = Objective(expr= model.z, sense=maximize)

# Constraints
model.con1 = Constraint(expr=sum(model.x[j] for j in model.I) == 1)

# Define the matrix A as a dictionary
A_values = {(1, 1): 4, (1, 2): -10, (1, 3): -10, (2, 1): -8, (2, 2): 5, (2, 3): -8, (3, 1): -12, (3, 2): -12, (3, 3): 9}
model.A = Param(model.I, model.I, initialize=A_values)

def cost_constraint_rule(model, i):
    return model.z - np.sum(model.A[i,j] * model.x[j] for j in model.I)<= 0

model.con2 = Constraint(model.I, rule=cost_constraint_rule)


Soving the above model using a solver( I used Gurobi)

In [6]:
solver = SolverFactory('gurobi')
solver.solve(model)

{'Problem': [{'Name': 'x1', 'Lower bound': -4.598130841121495, 'Upper bound': -4.598130841121495, 'Number of objectives': 1, 'Number of constraints': 4, 'Number of variables': 4, 'Number of binary variables': 0, 'Number of integer variables': 0, 'Number of continuous variables': 4, 'Number of nonzeros': 15, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Return code': '0', 'Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Wall time': '0.0', 'Error rc': 0, 'Time': 0.3342111110687256}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

Displaying the result

In [7]:
print("Optimal value of the objective function:", model.obj())
print("Optimal values of x:")
for j in model.I:
    print(f"x[{j}] =", value(model.x[j]))

Optimal value of the objective function: -4.598130841121495
Optimal values of x:
x[1] = 0.38584779706275035
x[2] = 0.2616822429906542
x[3] = 0.35246995994659547
